In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
import os
import mne
import numpy as np
import cortex
import plotly.graph_objects as go
from scipy import stats
from scipy.stats import wilcoxon
import plotly
import plotly.io as pio   
pio.kaleido.scope.mathjax = None
import statsmodels
subjects_dir = mne.datasets.sample.data_path() / 'subjects'
labels = mne.read_labels_from_annot(
    'fsaverage', 'HCPMMP1', 'both', subjects_dir=subjects_dir)
m = np.zeros(cortex.db.pycortex_fsaverage.surfaces.inflated.get()[0][0].shape[0])

Reading labels from parcellation...
   read 181 labels from /home/subbainria/mne_data/MNE-sample-data/subjects/fsaverage/label/lh.HCPMMP1.annot
   read 181 labels from /home/subbainria/mne_data/MNE-sample-data/subjects/fsaverage/label/rh.HCPMMP1.annot


In [3]:
for i in np.arange(360):
    print(i, labels[i])

0 <Label | fsaverage, '???-lh', lh : 13868 vertices>
1 <Label | fsaverage, '???-rh', rh : 13940 vertices>
2 <Label | fsaverage, 'L_10d_ROI-lh', lh : 867 vertices>
3 <Label | fsaverage, 'L_10pp_ROI-lh', lh : 401 vertices>
4 <Label | fsaverage, 'L_10r_ROI-lh', lh : 445 vertices>
5 <Label | fsaverage, 'L_10v_ROI-lh', lh : 694 vertices>
6 <Label | fsaverage, 'L_11l_ROI-lh', lh : 713 vertices>
7 <Label | fsaverage, 'L_13l_ROI-lh', lh : 791 vertices>
8 <Label | fsaverage, 'L_1_ROI-lh', lh : 2162 vertices>
9 <Label | fsaverage, 'L_23c_ROI-lh', lh : 1253 vertices>
10 <Label | fsaverage, 'L_23d_ROI-lh', lh : 511 vertices>
11 <Label | fsaverage, 'L_24dd_ROI-lh', lh : 1115 vertices>
12 <Label | fsaverage, 'L_24dv_ROI-lh', lh : 677 vertices>
13 <Label | fsaverage, 'L_25_ROI-lh', lh : 369 vertices>
14 <Label | fsaverage, 'L_2_ROI-lh', lh : 2554 vertices>
15 <Label | fsaverage, 'L_31a_ROI-lh', lh : 346 vertices>
16 <Label | fsaverage, 'L_31pd_ROI-lh', lh : 545 vertices>
17 <Label | fsaverage, 'L_31p

In [4]:
from voxelwise_tutorials.viz import plot_flatmap_from_mapper, _map_to_2d_cmap, plot_2d_flatmap_from_mapper
import numpy as np
import cortex
import os
import matplotlib.pyplot as plt
from voxelwise_tutorials.io import load_hdf5_sparse_array

In [8]:
early_visual_rois = ['V1','V2']
early_indices = [145, 146]

vwfa_indices = [  111, 138]
vwfa_rois = ['PH','TE2P']

early_auditory_rois = ['A1','LBelt','PBelt','MBelt','RI','A4']
early_auditory_indices = [55, 81, 96, 87, 123, 56]

primary_auditory_rois = ['A1']
primary_auditory_indices = [55]

secondary_auditory_rois = ['STV']
secondary_auditory_indices = [132]

late_language_rois = ['A5','44','45', 'IFJa', 'IFSp', 'PGi', 'PGp', 'PGs', 'TPOJ1', 'TPOJ2', 'TPOJ3', 'STGa','STSda', 'STSdp','TA2', 'STSva', 'STSvp']
late_language_indices = [57, 22, 23, 79, 81, 150, 143, 151, 139, 140, 144, 127, 128, 176, 107, 129, 130]

main_language_rois = ['EV','VWFA','EAC','LL','A1']
main_language_indices = [early_indices, vwfa_indices, early_auditory_indices, late_language_indices, primary_auditory_indices]

In [9]:
text_models_lan_residuals = ['res_numletters_bert','res_numphonemes_bert','res_numwords_bert','res_diphone_bert','gpt2-letters','gpt2-phonemes','gpt2-words','gpt2-diphone']
text_models_speech_residuals = ['bert-fbank','bert-mfcc', 'bert-mel', 'res_spectrpower_bert', 'res_phonological_bert', 'bert-articulation','gpt2-fbank','gpt2-mfcc', 'gpt2-mel', 'gpt2-powspec', 'res_phonological_bert','gpt2-articulation']
speech_models_lan_residuals = ['res_numletters_wav2vec','res_numphonemes_wav2vec','res_numwords_wav2vec','res_diphone_wav2vec', 'whisper-numletters','whisper-numphonemes','whisper-words']
speech_models_speech_residuals = ['wav2vec-fbank','wav2vec-mfcc', 'wav2vec-mel', 'res_powerspec_wav2vec','res_phonological_wav2vec','wav2vec-articulation', 'whisper-fbank','whisper-mfcc', 'whisper-mel', 'whisper-powspec','whisper-phonological','whisper-articulation']
text_models_vision_residuals = ['bert-motion']
speech_models_vision_residuals = ['wav2vec-motion']

In [10]:
normalized_scores_text_lan = {}
subjects = ['01','02','03','05','07','08']
for eachmodel in text_models_lan_residuals:
    normalized_scores_text_lan[eachmodel] = []
    for eachroi in np.arange(len(main_language_indices)):
        temp2 = []
        for subroi in main_language_indices[eachroi]:
            temp1 = []
            for eachsub in subjects:
                directory = '../mappers/'
                mapper_file = os.path.join(directory, f'subject{eachsub}_mappers.hdf')
                voxel_to_fsaverage = load_hdf5_sparse_array(mapper_file, key='voxel_to_fsaverage')
                ceiling_voxcorrs = np.load('/predictions_results/noise_ceiling/subject_'+str(eachsub)+'_kernel_ridge.npy')
                temp = []
                for eachlayer in np.arange(24):
                    try:
                        model_voxcorrs = np.load(eachmodel+'/listening/'+str(eachsub)+'/'+str(eachlayer)+'/'+"layer_"+str(eachlayer)+'.npy')
                        significant_indices = np.where(np.nan_to_num(ceiling_voxcorrs)<0.05)[0]
                        model_voxcorrs[significant_indices] = 0.0
                        normalize_across_voxels = np.nan_to_num(model_voxcorrs)/np.nan_to_num(ceiling_voxcorrs)
                        ev_projected_voxels = voxel_to_fsaverage @ normalize_across_voxels #163842
                        lh = ev_projected_voxels[labels[subroi].vertices] 
                        rh = ev_projected_voxels[labels[subroi+180].vertices+163842]
                        lh_rh = np.concatenate([lh,rh],axis=0)
                        temp.append(np.mean(lh_rh))
                    except:
                        continue
                temp1.append(np.mean(temp))
            temp2.append(np.array(temp1))
        normalized_scores_text_lan[eachmodel].append(np.mean(np.array(temp2),axis=0))
normalized_scores_text_lan = np.array(list(normalized_scores_text_lan.values()))
print(normalized_scores_text_lan.shape)

(8, 5, 6)
